In [14]:
import json
import os
import numpy as np
import shutil
import pandas as pd
import copy
main_directory="./results_real_data/"
subdirectories = ['results_FashionMNIST/', "results_FashionMNIST_CNN/", 'results_mnist/', 'results_MNIST_CNN/', 'results_News/', 'results_SVM/']#'results_FashionMNIST_recalibrator/', "results_FashionMNIST_CNN_recalibrator/", 'results_mnist_recalibrator/', 'results_MNIST_CNN_recalibrator/', 'results_News_recalibrator/', 'results_SVM_recalibrator/']
seed_count = {}
for subdirectory in subdirectories:
    full_path = os.path.join(main_directory, subdirectory)
    subdirect_count_complete = 0
    subdirect_missing = 0
    for foldername in os.listdir(full_path):
        folder = os.path.join(full_path, foldername)
        if os.path.isdir(folder):
            has_params = False
            has_metrics = False
            has_dataset = False
            for filename in os.listdir(folder):
                if filename.find("param") != -1:
                    params = json.load(open(os.path.join(folder, filename)))
                    if params['seed'] not in list(seed_count.keys()):
                        seed_count[params['seed']] = 1
                    else:
                        seed_count[params['seed']] += 1
                    has_params = True
                elif filename.find("metric") != -1:
                    has_metrics = True
                elif filename.find("dataset") != -1:
                    has_data = True
            if not has_params or not has_metrics or not has_data:
                
                subdirect_missing += 1
                    #if params['acquisition'] == "TS":
                #shutil.rmtree(folder)
            else:
                for tag in foldername.split("--"):
                    if tag.find("seed") != -1:
                        if int(tag.split("-")[1]) > 59:
                            shutil.rmtree(folder)
                subdirect_count_complete += 1
    


In [15]:
print(seed_count)
for key in seed_count.keys():
    if seed_count[key] != 96:
        print(key)
        #print(seed_count[key])

{20: 96, 21: 96, 30: 96, 31: 96, 50: 96, 51: 96, 22: 96, 23: 96, 32: 96, 33: 96, 52: 96, 53: 96, 24: 96, 25: 96, 34: 96, 35: 96, 54: 96, 55: 96, 26: 96, 27: 96, 36: 96, 37: 96, 56: 96, 57: 96, 28: 96, 29: 96, 38: 96, 39: 96, 58: 96, 59: 96, 6: 96, 7: 96, 8: 96, 9: 96, 10: 96, 11: 96, 12: 96, 13: 96, 14: 96, 15: 96, 16: 96, 17: 96, 18: 96, 19: 96, 40: 96, 41: 96, 42: 96, 43: 96, 44: 96, 45: 96, 46: 96, 47: 96, 48: 96, 49: 96, 1: 96, 2: 96, 3: 96, 4: 96, 5: 96}


In [27]:
#Instantaneous regret of pool rankings from real_data with vanilla uncertanties.
import json
import os
import numpy as np
import pandas as pd
import copy
exp_dict = {'surrogate': [], 'acquisition': [], 'seed':[], 'data':[], 'dist_nearest_train':[], 'inst_regret_test':[], 'inst_regret_pool':[], 'tot_regret_test':[], 'tot_regret_pool':[], 'calibration_mse':[], 'sharpness':[], 'x_opt_dist_test':[], 'x_opt_dist_pool':[]}
main_directory="./results_real_data/"
subdirectories = ['results_FashionMNIST/', "results_FashionMNIST_CNN/", 'results_mnist/', 'results_MNIST_CNN/', 'results_News/', 'results_SVM/']
for subdirectory in subdirectories:
    full_path = os.path.join(main_directory, subdirectory)
    for foldername in os.listdir(full_path):
        folder = os.path.join(full_path, foldername)
        if os.path.isdir(folder):
            for filename in os.listdir(folder):
                if filename.find("parameters") != -1:
                    json_file = open(os.path.join(folder, filename))
                    params = json.load(json_file)
                elif filename.find("metrics") != -1:
                    json_file = open(os.path.join(folder, filename))
                    metrics = json.load(json_file)
            if params['bo'] == True:
                exp_dict['surrogate'].append(params['surrogate'])
                exp_dict['acquisition'].append(params['acquisition'])
                exp_dict['data'].append(params['data_name'])
                exp_dict['dist_nearest_train'].append(metrics['next_sample_train_distance'])
                exp_dict['inst_regret_test'].append(metrics['y_regret_test'])
                exp_dict['inst_regret_pool'].append(metrics['y_regret_pool'])
                exp_dict['tot_regret_test'].append(np.cumsum(metrics['y_regret_test']))
                exp_dict['tot_regret_pool'].append(np.cumsum(metrics['y_regret_pool']))
                exp_dict['x_opt_dist_pool'].append(metrics['x_y_opt_dist_pool'])
                exp_dict['x_opt_dist_test'].append(metrics['x_y_opt_dist_test'])
                exp_dict['sharpness'].append(metrics['mean_sharpness'])
                exp_dict['calibration_mse'].append(metrics['y_calibration_mse'])
                exp_dict['seed'].append(params['seed'])
df = pd.DataFrame.from_dict(exp_dict)
processed_results = {'surrogate': [], 'acquisition': [], 'seed':[], 'data':[], 'dist_nearest_train_mean':[], 'inst_regret_test':[], 'inst_regret_pool':[], 'tot_regret_test':[], 'tot_regret_pool':[], 'calibration_mse':[], 'sharpness':[], 'x_opt_dist_test':[], 'x_opt_dist_pool':[]}
for index, row in df.iterrows():
    processed_results['surrogate'].append(row['surrogate'])
    processed_results['acquisition'].append(row['acquisition'])
    processed_results['data'].append(row['data'])
    processed_results['seed'].append(row['seed'])
    processed_results['dist_nearest_train_mean'].append(np.array(row['dist_nearest_train']).mean()) #Mean across one BO run.
    processed_results['inst_regret_pool'].append(np.array(row['inst_regret_pool'])[-1]) #Last iter.
    processed_results['inst_regret_test'].append(np.array(row['inst_regret_test'])[-1]) #Last iter.
    processed_results['tot_regret_test'].append(np.array(row['tot_regret_test'])[-1]) #Last iter.
    processed_results['tot_regret_pool'].append(np.array(row['tot_regret_pool'])[-1]) #Last iter.
    processed_results['calibration_mse'].append(np.array(row['calibration_mse']).mean()) #Mean Calibration MSE over run.
    processed_results['sharpness'].append(np.array(row['sharpness']).mean()) #Mean sharpness over run.
    processed_results['x_opt_dist_test'].append(np.array(row['x_opt_dist_test'])[-1])
    processed_results['x_opt_dist_pool'].append(np.array(row['x_opt_dist_pool'])[-1])
#Same seed and same dataset are ranked together.
df = pd.DataFrame.from_dict(processed_results)

In [37]:
print(set(df.loc[df['surrogate']=="RF"]['seed']))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 36, 38}
